<a href="https://colab.research.google.com/github/maegnji/XGBoost/blob/main/221024_ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 경로 설정 (공백있을시에 앞에 \필요)
%cd /content/drive/MyDrive/python/XGBoost/Chapter02

/content/drive/MyDrive/python/XGBoost/Chapter02


In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
df_census = pd.read_csv('census_cleaned.csv')

In [6]:
x = df_census.iloc[:,:-1]
y = df_census.iloc[:,-1]

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2)

In [9]:
# accuracy_score()는 맞는횟수 / 전체횟수
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [10]:
# 로지스틱회귀분석과 비슷하게 해주기 위하여 random_state=2로 세팅
clf = DecisionTreeClassifier(random_state=2)

In [11]:
clf.fit(x_train, y_train)

DecisionTreeClassifier(random_state=2)

In [12]:
y_pred = clf.predict(x_test)

In [13]:
# 동일한 데이터셋으로 1장에서 로지스틱 회귀의 정확도와 거의 비슷함
accuracy_score(y_pred, y_test)

0.8131679154894976